# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902043


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  13%|█▎        | 4/30 [00:03<00:21,  1.21it/s]

Rendering models:  17%|█▋        | 5/30 [00:08<00:52,  2.11s/it]

Rendering models:  20%|██        | 6/30 [00:09<00:41,  1.72s/it]

Rendering models:  23%|██▎       | 7/30 [00:10<00:36,  1.59s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:23<00:02,  1.35it/s]

Rendering models:  93%|█████████▎| 28/30 [00:24<00:01,  1.29it/s]

Rendering models:  97%|█████████▋| 29/30 [00:24<00:00,  1.37it/s]

Rendering models: 100%|██████████| 30/30 [00:25<00:00,  1.30it/s]

kayleebug2017                         0.003582
equidad1                              0.050348
mollollo                              0.002440
ElisabethB                            0.005935
not-logged-in-03b60317d6ee47942c58    0.000467
Thedunkmasta                          0.002354
Alexander00                           0.000357
not-logged-in-3a93234a2311f6df1afd    0.005842
not-logged-in-2a01249de2537c118533    0.213989
homant                                0.000534
pangeli5                              0.000371
Saharisunshine                        0.000523
acapirala                             0.000634
rhondacard                            0.001788
helios1235                            0.000866
Lavadude                              0.022105
Alissa18                              0.000851
Mkcls                                 0.000600
not-logged-in-490f77176b83d659546b    0.010314
ckkmomma                              0.000647
not-logged-in-5c973654fb0968b87ca1    0.000581
Its_Foster   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())